<a href="https://colab.research.google.com/github/aristocrat71/Bone.ai/blob/main/models/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import shutil
from glob import glob
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/MURA-v1.1.zip"
extract_to = "/content/drive/MyDrive"

In [ ]:
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
!mv "/content/MURA-v1.1" "/content/drive/MyDrive"

In [ ]:
!cp "/content/drive/MyDrive/MURA-v1.1/train" "/content/drive/MyDrive/MURA_Bone_Classes_train_flattened"

In [ ]:
base_dir = "/content/drive/MyDrive/MURA-v1.1/train"
target_dir = "/content/drive/MyDrive/MURA_Bone_Classes"

os.makedirs(target_dir, exist_ok=True)

for bone_type in os.listdir(base_dir):
    bone_path = os.path.join(base_dir, bone_type)
    if os.path.isdir(bone_path):
        class_dir = os.path.join(target_dir, bone_type)
        os.makedirs(class_dir, exist_ok=True)

        for patient_folder in os.listdir(bone_path):
            patient_path = os.path.join(bone_path, patient_folder)
            for study_folder in os.listdir(patient_path):
                study_path = os.path.join(patient_path, study_folder)
                image_files = glob(os.path.join(study_path, "*.png"))

                for img_file in image_files:
                    new_filename = f"{bone_type}_{patient_folder}_{study_folder}_{os.path.basename(img_file)}"
                    shutil.copy(img_file, os.path.join(class_dir, new_filename))

In [ ]:
samp_img = "/content/drive/MyDrive/MURA_Bone_Classes/XR_ELBOW/XR_ELBOW_patient00011_study1_negative_image1.png"
img = Image.open(samp_img)

plt.imshow(img)
plt.show()

In [ ]:
! pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 572.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image

# Custom function to verify images before loading
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
            # Also try to load it
            with Image.open(file_path) as img:
                img.load()
        return True
    except:
        return False

# Filter function for flow_from_directory
def valid_image_filter(directory, subdirs, files):
    # Only keep files that can be successfully loaded
    file_paths = [os.path.join(directory, f) for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    valid_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg')) and is_valid_image(os.path.join(directory, f))]
    return valid_files

# Create data generators with image validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    # Add filter function for corrupted images
    validation_split=0.2
)

# For body part classification with validation filtering
def create_body_part_generators(batch_size=32, img_size=(224, 224)):
    body_parts = ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND',
                 'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']

    # First scan and filter directories
    train_path = '/content/drive/MyDrive/MURA-v1.1/train'
    valid_path = '/content/drive/MyDrive/MURA-v1.1/valid'

    train_generator = datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        classes=body_parts,
        shuffle=True,
        subset='training',
        # Use try/except in case your version doesn't support this
        # ImageDataGenerator may not natively support this without modification
    )

    valid_generator = datagen.flow_from_directory(
        valid_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        classes=body_parts,
        shuffle=False,
        subset='validation',
    )

    return train_generator, valid_generator


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def build_mura_bodypart_classifier(img_size=(224, 224)):
    # Load DenseNet169 with pre-trained ImageNet weights
    base_model = DenseNet169(
        weights='imagenet',
        include_top=False,
        input_shape=(img_size[0], img_size[1], 3)
    )

    # Add custom classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)  # Strong regularization
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # 7 classes for body parts
    predictions = Dense(7, activation='softmax')(x)

    # Create the full model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base model layers (for initial training)
    for layer in base_model.layers:
        layer.trainable = False

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Create model
model = build_mura_bodypart_classifier()

# Create callbacks for training
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# Generate the training and validation data
train_generator, valid_generator = create_body_part_generators(batch_size=32)



51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 29452 images belonging to 7 classes.
Found 637 images belonging to 7 classes.


In [ ]:
train_generator

In [ ]:
# Train the model with frozen layers first
history_frozen = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=callbacks
)

# Unfreeze some of the deeper layers for fine-tuning
for layer in model.layers[1].layers[-50:]:  # Unfreeze the last 50 layers of DenseNet
    layer.trainable = True

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with unfrozen layers
history_unfrozen = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=40,
    callbacks=callbacks,
    initial_epoch=5
)

# Saving the model
model.save('mura_bodypart_classifier.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 75/921 ━━━━━━━━━━━━━━━━━━━━ 3:24:22 14s/step - accuracy: 0.5449 - loss: 1.4785

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7aa96c5ec630>
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 248, in _finite_generator
    yield self._standardize_ba

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('mura_bodypart_classifier.h5')


In [ ]:
_, valid_generator = create_body_part_generators(batch_size=32)
loss, accuracy = model.evaluate(valid_generator, verbose=1)
print(f"Validation loss: {loss:.4f}")
print(f"Validation accuracy: {accuracy:.4f}")

In [ ]:
import numpy as np

# Get true labels and predictions
y_true = valid_generator.classes
y_pred = model.predict(valid_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Example: Compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)
